ใส่ path raw_data ที่ input_dir 
ใส่ path output ที่ output_dir
code นี้ จะทำการดึงข้อมูลจาก raw_data มาจัดโครงสร้างตามรูปแบบของอุปกรณ์ TPL

In [41]:
import os
import re
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed

def copy_files(src_dst_pairs):
    for src_file, dst_file in src_dst_pairs:
        shutil.copy2(src_file, dst_file)

def rename_files(file_pairs):
    for src_file, new_file_path in file_pairs:
        os.rename(src_file, new_file_path)

def process_date_folder(date_folder_name, input_dir, output_dir):
    # Create the main folder name
    main_folder_name = f"survey_data_{date_folder_name}"

    # Create the full path for the new directory
    survey_data_path = os.path.join(output_dir, main_folder_name)
    # Create the new directory
    os.makedirs(survey_data_path, exist_ok=True)

    # Create subdirectories
    subdirectories = ['Data', 'Output', 'PAVE', 'ROW']
    for subdirectory in subdirectories:
        subdirectory_path = os.path.join(survey_data_path, subdirectory)
        os.makedirs(subdirectory_path, exist_ok=True)

    # Paths for source directories
    data_folder_path = os.path.join(input_dir, date_folder_name, 'data')
    output_path = os.path.join(survey_data_path, 'Output')
    data_path = os.path.join(survey_data_path, 'Data')
    photo_directory = os.path.join(input_dir, date_folder_name, 'photo', date_folder_name)

    # Check if the Data directory exists for the current date
    if os.path.exists(data_folder_path):
        for folder_name in os.listdir(data_folder_path):
            folder_path = os.path.join(data_folder_path, folder_name)
            if os.path.isdir(folder_path):
                # Create each folder found in the Data directory inside the Output directory
                output_folder_path = os.path.join(output_path, folder_name)
                os.makedirs(output_folder_path, exist_ok=True)

    # Copy .xlsx files from the source directory to the Output subdirectory
    xlsx_files = []
    for root, dirs, files in os.walk(input_dir):
        for file_name in files:
            if file_name.endswith('.xlsx'):
                src_file = os.path.join(root, file_name)
                # Check if the .xlsx file belongs to the current date folder
                if date_folder_name in root:
                    dst_file = os.path.join(output_path, file_name)
                    xlsx_files.append((src_file, dst_file))
    copy_files(xlsx_files)
    
    for data_output in os.listdir(data_folder_path):
        data_output_path = os.path.join(data_folder_path, data_output)
        if os.path.isdir(data_output_path):
            data_number = data_output.replace(date_folder_name, "").replace("RUN", "").lstrip("0")
            new_folder_data = f"{date_folder_name}_{data_number}"
            new_folder_data_path = os.path.join(data_path,new_folder_data)
            os.makedirs(new_folder_data_path, exist_ok=True)
   
   #hi
    #Process Camera_GeoTagged and Log directories for the current date folder
    for run_folder in os.listdir(data_folder_path):
        run_folder_path = os.path.join(data_folder_path, run_folder)
        if os.path.isdir(run_folder_path):
            # Process Camera_GeoTagged
            camera_geotagged_path = os.path.join(run_folder_path, 'Camera_GeoTagged')
            if os.path.exists(camera_geotagged_path):
                run_number = run_folder.replace(date_folder_name, "").replace("RUN", "").lstrip("0")
                new_folder_name = f"{date_folder_name}_{run_number}"
                new_folder_path = os.path.join(survey_data_path, 'PAVE', new_folder_name, 'PAVE-0')
                os.makedirs(new_folder_path, exist_ok=True)

                # Copy .jpg files to the new folder and rename them
                jpg_files = []
                renamed_files = []
                jpg_counter = 1
                for file_name in os.listdir(camera_geotagged_path):
                    if file_name.endswith('.jpg'):
                        src_file = os.path.join(camera_geotagged_path, file_name)
                        dst_file = os.path.join(new_folder_path, file_name)
                        jpg_files.append((src_file, dst_file))

                        # Rename the file
                        new_file_name = f"{date_folder_name}_{run_number}-PAVE-0-{jpg_counter:05d}.jpg"
                        new_file_path = os.path.join(new_folder_path, new_file_name)
                        renamed_files.append((dst_file, new_file_path))
                        jpg_counter += 1

                copy_files(jpg_files)
                rename_files(renamed_files)

  ## Process Log
            log_path = os.path.join(run_folder_path, 'Log')
            if os.path.exists(log_path):
                for file_name in os.listdir(log_path):
                    if file_name.endswith(f'{run_folder}.csv'):
                        csv_path = os.path.join(log_path, file_name)
                        destination_subfolder_path = os.path.join(output_path, run_folder)
                        os.makedirs(destination_subfolder_path, exist_ok=True)
                        shutil.copy2(csv_path, destination_subfolder_path)

    # ## Process ROW directory within photo directory
    if os.path.exists(photo_directory):
        for photo_run_folder in os.listdir(photo_directory):
            photo_run_folder_path = os.path.join(photo_directory, photo_run_folder)
            if os.path.isdir(photo_run_folder_path):
                # Process ROW
                run_number = photo_run_folder.replace(date_folder_name, "").replace("RUN", "").lstrip("0")
                new_folder_name = f"{date_folder_name}_{run_number}"
                new_folder_path = os.path.join(survey_data_path, 'ROW', new_folder_name, 'ROW-0')
                os.makedirs(new_folder_path, exist_ok=True)

                # Copy .jpg files to the new folder and rename them
                jpg_files = []
                renamed_files = []
                jpg_counter = 1
                for file_name in os.listdir(photo_run_folder_path):
                    if file_name.endswith('.jpg'):
                        src_file = os.path.join(photo_run_folder_path, file_name)
                        dst_file = os.path.join(new_folder_path, file_name)
                        jpg_files.append((src_file, dst_file))

                        # Ensure unique file name
                        new_file_name = f"{date_folder_name}_{run_number}-ROW-0-{jpg_counter:05d}.jpg"
                        new_file_path = os.path.join(new_folder_path, new_file_name)
                        renamed_files.append((dst_file, new_file_path))
                        jpg_counter += 1

                copy_files(jpg_files)
                rename_files(renamed_files)

def copy_and_organize_files(input_dir, output_dir):
    # Create the destination parent directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Find all date folders in the source directory
    date_folders = [folder_name for folder_name in os.listdir(input_dir) if re.match(r'^\d{8}$', folder_name)]

    if not date_folders:
        print("No date folders found in the source directory.")
    else:
        with ThreadPoolExecutor(max_workers=100) as executor:
            future_to_date_folder = {executor.submit(process_date_folder, date_folder_name, input_dir, output_dir): date_folder_name for date_folder_name in date_folders}

            for future in as_completed(future_to_date_folder):
                date_folder_name = future_to_date_folder[future]
                try:
                    future.result()
                    print(f"✅ Processed folder: {date_folder_name} Successfully")
                except Exception as exc:
                    print(f"{date_folder_name} generated an exception: {exc}")

# def main():
#     ghjg

    
if __name__ == "__main__":
    base_dir = r"D:\test_xeno\123\xenomatix"
    input_dir = os.path.join(base_dir, "input")
    output_dir = os.path.join(base_dir, "output")
    copy_and_organize_files(input_dir, output_dir)






AttributeError: module 'os' has no attribute 'join'